In [1]:
# CODE WRITING BY LINDSAY - THANK YOU

from caveclient import CAVEclient
from datetime import datetime

cave_client_public = CAVEclient('minnie65_public_v117')
cave_client = CAVEclient('minnie65_phase3_v1')

# get timestamp for public release
past_timestamp = cave_client.materialize.get_timestamp(117)

# get old nuclei classification table
nuclei_before = cave_client_public.materialize.query_table('nucleus_neuron_svm')

# get new nuclei classification table
nuclei_now = cave_client.materialize.query_table('allen_neuron_nonneuron_svm_v0',
                                                timestamp=datetime.utcnow())

# filter to just single soma
single_soma_after = nuclei_now['pt_root_id'].value_counts()[nuclei_now['pt_root_id'].value_counts() < 2]
single_soma_df_after = nuclei_now[nuclei_now['pt_root_id'].isin(single_soma_after.keys())]

# merge old and new, keeping only nuclei in the new table
# new table has already filtered out nuclei on the edge and in bad data areas
nuclei_table = nuclei_before[['id', 'pt_root_id', 'pt_position']].merge(
    single_soma_df_after[['id', 'cell_type']], on='id', how='right')

# get just neurons
neurons = nuclei_table[nuclei_table.cell_type == 'neuron']

In [ ]:
queue_url = sqs.get_or_create_queue("Root_ids")
running_list = []
for i, root_id in enumerate(neurons.pt_root_id.unique()[:100]):
    running_list.append({
            'Id': str(i),
            'MessageBody': str(root_id),
        })
    if i % 10 == 0:
        sqs.send_batch(queue_url, running_list)
        running_list = []

In [2]:
neurons

,id,pt_root_id,pt_position,cell_type
2,204710,864691135497604883,"[134208, 247184, 18598]",neuron
3,485509,864691136740606812,"[282608, 103808, 20318]",neuron
4,113721,864691135366988025,"[110208, 153664, 23546]",neuron
5,263203,864691135181741826,"[166512, 174176, 24523]",neuron
6,456177,864691135337690598,"[275616, 135120, 24873]",neuron
...,...,...,...,...
83619,394515,864691136050803443,"[229504, 152816, 20596]",neuron
83620,390522,864691135939301377,"[239920, 112864, 17348]",neuron
83621,329229,864691135740183915,"[202176, 151760, 15660]",neuron
83623,256280,864691136266562036,"[165808, 118256, 20198]",neuron
